![image.png](https://miro.medium.com/max/1400/1*5rairMYDqDpTmzytnqpYnQ.jpeg)
[Medium](https://medium.com/@naivetech/the-connection-between-spark-streaming-and-apache-kafka-with-java-9fcdc701a8ad)

## Spark Streaming + Kafka Integration Guide (Kafka broker version 0.10.0 or higher
[Link](https://spark.apache.org/docs/latest/streaming-kafka-0-10-integration.html)


The Spark Streaming integration for Kafka 0.10 provides simple parallelism, 1:1 correspondence between Kafka partitions and Spark partitions, and access to offsets and metadata. However, because the newer integration uses the new [Kafka consumer API](https://kafka.apache.org/documentation.html#newconsumerapi) instead of the simple API, there are notable differences in usage.

### Linking
This approach is supported only in Scala/Java application. Link your SBT/Maven project with the following artifact (see Linking section in the main programming guide for further information).

```properties
groupId = org.apache.spark
artifactId = spark-streaming-kafka-0-10_2.12
version = 3.2.1
 ```

# An example in Java

```bash
# Compile
cd Dev/tap/spark/apps/TapSparkDemo
mvn build
# Start Zookeeper
./kafkaStartZk.sh
# Start Kafka Server
./kafkaStartServer.sh
#  Start Spark Streaming
./sparkSubmitApps.sh TapSpark TapSparkDemo/target/simple-project-1.0.jar org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.1
# Start Producer
./flumeTwitterKafka.sh
```

![](https://i.imgflip.com/57d4er.jpg)
[NicsMeme](https://imgflip.com/i/57d4er)

## Structured Streaming + Kafka Integration Guide (Kafka broker version 0.10.0 or higher)
[Link](https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html)


Structured Streaming integration for Kafka 0.10 to read data from and write data to Kafka.

## Linking
For Scala/Java applications using SBT/Maven project definitions, link your application with the following artifact:

```properties
groupId = org.apache.spark
artifactId = spark-sql-kafka-0-10_2.12
version = 3.2.1
```

Please note that to use the headers functionality, your Kafka client version should be version 0.11.0.0 or up.

For Python applications, you need to add this above library and its dependencies when deploying your application. See the Deploying subsection below.

For experimenting on spark-shell, you need to add this above library and its dependencies too when invoking spark-shell. Also, see the Deploying subsection below.

## Example

### PythonStructuredStreamingKafka

```bash
# Start Zookeeper
./kafkaStartZk.sh
# Start Kafka Server
./kafkaStartServer.sh
#  Start Spark Streaming
./sparkSubmitPython.sh kafka_structuredstream.py org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1
# Start Producer
./kafkaCreateProducer.sh
```

## Creating a Kafka Source

### Streaming Queries

Standard approach, connects Kafka topics (one or more) and run queries

```python
# Subscribe to 1 topic
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2") \
  .option("subscribe", "topic1") \
  .load()
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
```

### Batch Queries

If you have a use case that is better suited to batch processing, you can create a Dataset/DataFrame for a defined range of offsets.



```python
df = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2") \
  .option("subscribe", "topic1") \
  .load()
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
```

## Row format

| Column             | Type      |
|--------------------|-----------|
| key                | binary    |
| value              | binary    |
| topic              | string    |
| partition          | int       |
| offset             | long      |
| timestamp          | timestamp |
| timestampType      | int       |
| headers (optional) | array     |

## Kafka Required Options

| Option                  | value                                       | meaning                                                                                                                                      |
|-------------------------|---------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------|
| assign                  | json string {"topicA":[0,1],"topicB":[2,4]} | Specific TopicPartitions to consume. Only one of "assign", "subscribe" or "subscribePattern" options can be specified for Kafka source.      |
| subscribe               | A comma-separated list of topics            | The topic list to subscribe. Only one of "assign", "subscribe" or "subscribePattern" options can be specified for Kafka source.              |
| subscribePattern        | Java regex string                           | The pattern used to subscribe to topic(s). Only one of "assign, "subscribe" or "subscribePattern" options can be specified for Kafka source. |
| kafka.bootstrap.servers | A comma-separated list of host:port         | The Kafka "bootstrap.servers" configuration.                                                                                                 |

## Kafka Optional Options

| Option                      | value                                                                                                      | default                                        | query type          | meaning                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|-----------------------------|------------------------------------------------------------------------------------------------------------|------------------------------------------------|---------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| startingOffsetsByTimestamp  | json string """ {"topicA":{"0": 1000, "1": 1000}, "topicB": {"0": 2000, "1": 2000}} """                    | none (the value of startingOffsets will apply) | streaming and batch | The start point of timestamp when a query is started, a json string specifying a starting timestamp for each TopicPartition. The returned offset for each partition is the earliest offset whose timestamp is greater than or equal to the given timestamp in the corresponding partition. If the matched offset doesn't exist, the query will fail immediately to prevent unintended read from such partition. (This is a kind of limitation as of now, and will be addressed in near future.)  Spark simply passes the timestamp information to KafkaConsumer.offsetsForTimes, and doesn't interpret or reason about the value. For more details on KafkaConsumer.offsetsForTimes, please refer javadoc for details. Also the meaning of timestamp here can be vary according to Kafka configuration (log.message.timestamp.type): please refer Kafka documentation for further details. Note: This option requires Kafka 0.10.1.0 or higher. Note2: startingOffsetsByTimestamp takes precedence over startingOffsets. Note3: For streaming queries, this only applies when a new query is started, and that resuming will always pick up from where the query left off. Newly discovered partitions during a query will start at earliest. |
| startingOffsets             | "earliest", "latest" (streaming only), or json string """ {"topicA":{"0":23,"1":-1},"topicB":{"0":-2}} """ | "latest" for streaming, "earliest" for batch   | streaming and batch | The start point when a query is started, either "earliest" which is from the earliest offsets, "latest" which is just from the latest offsets, or a json string specifying a starting offset for each TopicPartition. In the json, -2 as an offset can be used to refer to earliest, -1 to latest. Note: For batch queries, latest (either implicitly or by using -1 in json) is not allowed. For streaming queries, this only applies when a new query is started, and that resuming will always pick up from where the query left off. Newly discovered partitions during a query will start at earliest.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| endingOffsetsByTimestamp    | json string """ {"topicA":{"0": 1000, "1": 1000}, "topicB": {"0": 2000, "1": 2000}} """                    | latest                                         | batch query         | The end point when a batch query is ended, a json string specifying an ending timestamp for each TopicPartition. The returned offset for each partition is the earliest offset whose timestamp is greater than or equal to the given timestamp in the corresponding partition. If the matched offset doesn't exist, the offset will be set to latest.  Spark simply passes the timestamp information to KafkaConsumer.offsetsForTimes, and doesn't interpret or reason about the value. For more details on KafkaConsumer.offsetsForTimes, please refer javadoc for details. Also the meaning of timestamp here can be vary according to Kafka configuration (log.message.timestamp.type): please refer Kafka documentation for further details. Note: This option requires Kafka 0.10.1.0 or higher. Note2: endingOffsetsByTimestamp takes precedence over endingOffsets.                                                                                                                                                                                                                                                                                                                                                                    |
| endingOffsets               | latest or json string {"topicA":{"0":23,"1":-1},"topicB":{"0":-1}}                                         | latest                                         | batch query         | The end point when a batch query is ended, either "latest" which is just referred to the latest, or a json string specifying an ending offset for each TopicPartition. In the json, -1 as an offset can be used to refer to latest, and -2 (earliest) as an offset is not allowed.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| failOnDataLoss              | true or false                                                                                              | true                                           | streaming and batch | Whether to fail the query when it's possible that data is lost (e.g., topics are deleted, or offsets are out of range). This may be a false alarm. You can disable it when it doesn't work as you expected.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| kafkaConsumer.pollTimeoutMs | long                                                                                                       | 120000                                         | streaming and batch | The timeout in milliseconds to poll data from Kafka in executors. When not defined it falls back to spark.network.timeout.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| fetchOffset.numRetries      | int                                                                                                        | 3                                              | streaming and batch | Number of times to retry before giving up fetching Kafka offsets.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| fetchOffset.retryIntervalMs | long                                                                                                       | 10                                             | streaming and batch | milliseconds to wait before retrying to fetch Kafka offsets                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| maxOffsetsPerTrigger        | long                                                                                                       | none                                           | streaming and batch | Rate limit on maximum number of offsets processed per trigger interval. The specified total number of offsets will be proportionally split across topicPartitions of different volume.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| minPartitions               | int                                                                                                        | none                                           | streaming and batch | Desired minimum number of partitions to read from Kafka. By default, Spark has a 1-1 mapping of topicPartitions to Spark partitions consuming from Kafka. If you set this option to a value greater than your topicPartitions, Spark will divvy up large Kafka partitions to smaller pieces. Please note that this configuration is like a hint: the number of Spark tasks will be approximately minPartitions. It can be less or more depending on rounding errors or Kafka partitions that didn't receive any new data.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| groupIdPrefix               | string                                                                                                     | spark-kafka-source                             | streaming and batch | Prefix of consumer group identifiers (group.id) that are generated by structured streaming queries. If "kafka.group.id" is set, this option will be ignored.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| kafka.group.id              | string                                                                                                     | none                                           | streaming and batch | The Kafka group id to use in Kafka consumer while reading from Kafka. Use this with caution. By default, each query generates a unique group id for reading data. This ensures that each Kafka source has its own consumer group that does not face interference from any other consumer, and therefore can read all of the partitions of its subscribed topics. In some scenarios (for example, Kafka group-based authorization), you may want to use a specific authorized group id to read data. You can optionally set the group id. However, do this with extreme caution as it can cause unexpected behavior. Concurrently running queries (both, batch and streaming) or sources with the same group id are likely interfere with each other causing each query to read only part of the data. This may also occur when queries are started/restarted in quick succession. To minimize such issues, set the Kafka consumer session timeout (by setting option "kafka.session.timeout.ms") to be very small. When this is set, option "groupIdPrefix" will be ignored.                                                                                                                                                                  |
| includeHeaders              | boolean                                                                                                    | false                                          | streaming and batch | Whether to include the Kafka headers in the row.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |

## Kafka Specific Configurations
Kafka’s own configurations can be set via DataStreamReader.option with kafka. prefix, e.g, stream.option("kafka.bootstrap.servers", "host:port"). For possible kafka parameters, see Kafka consumer config docs for parameters related to reading data, and Kafka producer config docs for parameters related to writing data.

Note that the following Kafka params cannot be set and the Kafka source or sink will throw an exception:

- group.id: Kafka source will create a unique group id for each query automatically. The user can set the prefix of the automatically generated group.id’s via the optional source option groupIdPrefix, default value is “spark-kafka-source”. You can also set “kafka.group.id” to force Spark to use a special group id, however, please read warnings for this option and use it with caution.
- auto.offset.reset: Set the source option startingOffsets to specify where to start instead. Structured Streaming manages which offsets are consumed internally, rather than rely on the kafka Consumer to do it. This will ensure that no data is missed when new topics/partitions are dynamically subscribed. Note that startingOffsets only applies when a new streaming query is started, and that resuming will always pick up from where the query left off.
- key.deserializer: Keys are always deserialized as byte arrays with ByteArrayDeserializer. Use DataFrame operations to explicitly deserialize the keys.
- value.deserializer: Values are always deserialized as byte arrays with ByteArrayDeserializer. Use DataFrame operations to explicitly deserialize the values.
- key.serializer: Keys are always serialized with ByteArraySerializer or StringSerializer. Use DataFrame operations to explicitly serialize the keys into either strings or byte arrays.
- value.serializer: values are always serialized with ByteArraySerializer or StringSerializer. Use DataFrame operations to explicitly serialize the values into either strings or byte arrays.
- enable.auto.commit: Kafka source doesn’t commit any offset.
- interceptor.classes: Kafka source always read keys and values as byte arrays. It’s not safe to use ConsumerInterceptor as it may break the query.

## Deploying

As with any Spark applications, spark-submit is used to launch your application. spark-sql-kafka-0-10_2.12 and its dependencies can be directly added to spark-submit using --packages, such as,

```bash
./bin/spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1 ...
```

For experimenting on spark-shell, you can also use --packages to add spark-sql-kafka-0-10_2.12 and its dependencies directly,

```bash
./bin/spark-shell --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1 ...
```

# Biblio

* https://www.slideshare.net/databricks/writing-continuous-applications-with-structured-streaming-python-apis-in-apache-spark
* https://kontext.tech/column/streaming-analytics/475/spark-structured-streaming-read-from-and-write-into-kafka-topics
* https://medium.com/knoldus/kafka-and-spark-streams-the-happily-ever-after-84b6f4b6b509